In [59]:
from pathlib import Path
import re

re_seeds = re.compile(r"seeds: ([\d\s]+)\n\n")
re_s2s = re.compile(r"seed-to-soil map:\n([\d\s]+)\n\n", re.MULTILINE)
re_s2f = re.compile(r"soil-to-fertilizer map:\n([\d\s]+)\n\n", re.MULTILINE)
re_f2w = re.compile(r"fertilizer-to-water map:\n([\d\s]+)\n\n", re.MULTILINE)
re_w2l = re.compile(r"water-to-light map:\n([\d\s]+)\n\n", re.MULTILINE)
re_l2t = re.compile(r"light-to-temperature map:\n([\d\s]+)\n\n", re.MULTILINE)
re_t2h = re.compile(r"temperature-to-humidity map:\n([\d\s]+)\n\n", re.MULTILINE)
re_h2l = re.compile(r"humidity-to-location map:\n([\d\s]+)\n", re.MULTILINE)

def get_rangs(inp: str):
    rangs = []
    for rang in inp.split("\n"):
        rangs.append([int(x) for x in rang.split(" ")])
    return rangs

def map_rangs(seeds, rangs, seed_ranges=False):
    res = []
    if not seed_ranges:
        for seed in seeds:
            for d, s, l in rangs:
                if seed >= s and seed <= s + l:
                    print(seed, s, d, l)
                    res.append(seed - s + d)
                    break
            else:
                res.append(seed)
        return res
    else:
        # print(rangs)
        tmp = []
        while seeds:
            ss = seeds.pop(0)
            sl = seeds.pop(0)
            for d, s, l in rangs:
                if ss >= s and ss < s + l:
                    if ss + sl > s + l:
                        tmp += [ss, s+l-ss]
                        seeds += [s+l, sl - tmp[-1]]
                        break
            else:
                tmp += [ss, sl]
        res =  []
        # print(tmp)
        for i in range(0, len(tmp), 2):
            ss, sl = tmp[i], tmp[i+1]
            for d, s, l in rangs:
                if ss >= s and ss < s + l:
                    res += [ss-s+d, sl]
                    break
            else:
                res += [ss, sl]
        # print("====")
        return res

def parse_map(path: str, seed_ranges=False):
    with Path(path).open() as f:
        data = f.read()
#        print(data)
        match = re_seeds.search(data)
        seeds = [int(s) for s in match.group(1).split(" ") if s]          
        # print(seeds)
        seeds = map_rangs(seeds, get_rangs(re_s2s.search(data).group(1)), seed_ranges)   
        # print(seeds)
        seeds = map_rangs(seeds, get_rangs(re_s2f.search(data).group(1)), seed_ranges)    
        # print(seeds)
        seeds = map_rangs(seeds, get_rangs(re_f2w.search(data).group(1)), seed_ranges)    
        # print(seeds)
        seeds = map_rangs(seeds, get_rangs(re_w2l.search(data).group(1)), seed_ranges)    
        # print(seeds)
        seeds = map_rangs(seeds, get_rangs(re_l2t.search(data).group(1)), seed_ranges)
        # print(seeds)
        seeds = map_rangs(seeds, get_rangs(re_t2h.search(data).group(1)), seed_ranges)
        # print(seeds)
        seeds = map_rangs(seeds, get_rangs(re_h2l.search(data).group(1)), seed_ranges)
        # print(seeds)
        if seed_ranges:
            return min(seeds[::2])
        return min(seeds)

In [60]:
asserparse_map("./05_test.txt", True) == 46

46

In [61]:
assert parse_map("./05_test.txt") == 35

79 50 52 48
55 50 52 48
14 0 39 15
13 0 39 15
53 53 49 8
57 53 49 8
52 11 0 42
81 25 18 70
49 25 18 70
53 25 18 70
41 25 18 70
74 64 68 13
46 45 81 19
42 0 1 69
34 0 1 69
78 56 60 37
82 56 60 37


In [57]:
assert parse_map("./05_input.txt") == 486613012

2906961955 2574476703 190915687 438548691
52237479 0 886579086 430196980
1600322402 1574488136 0 141789716
372221628 0 886579086 430196980
2347782594 1961915290 2568500570 563435442
164705568 0 886579086 430196980
541904540 430196980 1435686684 892701921
89745770 0 886579086 430196980
126821306 0 886579086 430196980
192539923 0 886579086 430196980
3411274151 3056742994 3183744888 1057221520
496169308 430196980 1435686684 892701921
919015581 430196980 1435686684 892701921
8667739 0 886579086 430196980
654599767 430196980 1435686684 892701921
160781040 0 886579086 430196980
3945616935 3056742994 3183744888 1057221520
85197451 0 886579086 430196980
999146581 430196980 1435686684 892701921
344584779 0 886579086 430196980
523400939 347687930 702170285 213620609
938816565 847324382 0 159395290
25834266 0 1772194633 170678435
1258800714 1006719672 1284811016 253212372
2954367874 2792977617 3187826244 352243190
1051284654 1006719672 1284811016 253212372
1547394244 1412279682 477747447 22156002

In [58]:
parse_map("./05_input.txt", True)

[2906961955, 52237479, 1600322402, 115955450, 2347782594, 164705568, 541904540, 89745770, 126821306, 192539923, 3411274151, 496169308, 919015581, 8667739, 654599767, 160781040, 3945616935, 85197451, 999146581, 323752320, 1716277852, 240111965, 1322898901, 20832459, 1956389817, 5525473, 1961915290, 10628740]
[523400939, 37907600, 25834266, 115955450, 2954367874, 164705568, 1547394244, 86445458, 1013400392, 192539923, 3538276045, 153382742, 1924505285, 8667739, 1660089471, 55573838, 4072618829, 44920119, 2004636285, 80047146, 2328388605, 74646983, 629464378, 20832459, 881053613, 5525473, 2568500570, 10628740, 561308539, 14329879, 1633839702, 3300312, 3691658787, 56026212, 1715663309, 105207202, 4117538948, 40277332, 2084683431, 243705174, 2403035588, 165464982, 3747684999, 21637270, 3769322269, 265123084]
[877883294, 37907600, 1798028899, 16308968, 3349216501, 125225795, 612862009, 19768751, 1291491736, 114007411, 3702046779, 153382742, 1124632870, 8667739, 1716620795, 55573838, 41803633

56931769